In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from freedom.utils.dataset import Data
from freedom.utils.callback import Save

from freedom.neural_nets.stringnet import get_stringnet

In [2]:
data = Data(dirs=['/home/iwsatlas1/peller/work/oscNext/level3_v01.03/140000_i3cols']) #,'/home/iwsatlas1/peller/work/oscNext/level3_v01.03/120000_i3cols'])

In [3]:
stringnet = get_stringnet(data.labels)

In [4]:
stringnet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
stringnet_trafo (stringnet_traf (None, 14)           0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           480         stringnet_trafo[0][0]            
______________________________________________________________________________________________

In [5]:
optimizer = tf.keras.optimizers.Adam(0.0001)
stringnet.compile(loss='binary_crossentropy', optimizer=optimizer)

In [6]:
train_id = 'StringNet_logE_' + datetime.datetime.now().strftime("%d_%b_%Y-%Hh%M")

In [16]:
callbacks = []
callbacks.append(Save(save_every=2, path_template='../../resources/models/'+train_id+'/epoch_%i', ))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='../../resources/logs/'+train_id, histogram_freq=1))

In [14]:
train_data, test_data = data.get_stringnet_data(train_batch_size=2**14,
                                                test_batch_size=2**14,
                                                shuffle_block_size=2**16
                                                )

In [15]:
d_train_data = train_data.apply(tf.data.experimental.prefetch_to_device(':'.join(tf.config.list_physical_devices(device_type="GPU")[0].name.split(':')[1:])))

In [ ]:
hist = stringnet.fit(d_train_data, epochs=1000, verbose=2, callbacks=callbacks, validation_data=test_data,
                     initial_epoch=len(callbacks[0].hist['train_losses']))

Epoch 1/1000
13066/13066 - 865s - loss: 0.6893 - val_loss: 0.6811
Epoch 2/1000
13066/13066 - 879s - loss: 0.6787 - val_loss: 0.6748
Epoch 3/1000
13066/13066 - 906s - loss: 0.6726 - val_loss: 0.6685
Epoch 4/1000
13066/13066 - 876s - loss: 0.6694 - val_loss: 0.6669
Epoch 5/1000
13066/13066 - 881s - loss: 0.6684 - val_loss: 0.6661
Epoch 6/1000
13066/13066 - 890s - loss: 0.6672 - val_loss: 0.6652
Epoch 7/1000


In [ ]:
# L3 14: 